In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4c7dda1382bf8a70c4f55f477b02f838f8e702a31c6998a4fde6dc6f3533e519
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col,lit

In [ ]:
spark=SparkSession.builder.appName("part2").getOrCreate()

In [ ]:
testDf=[[1,"jan"],[2,"feb"],[1,"jan"],[3,"may"],[4,"april"],[5,"aug"],[6,"may"],[6,"may"],[3,"may"],[12,"dec"]]

In [ ]:
schema=StructType([StructField("ID",IntegerType()),StructField("Month",StringType())])

In [ ]:
df_new=spark.createDataFrame(testDf,schema=schema)

In [ ]:
df_new.show()

+---+-----+
| ID|Month|
+---+-----+
|  1|  jan|
|  2|  feb|
|  1|  jan|
|  3|  may|
|  4|april|
|  5|  aug|
|  6|  may|
|  6|  may|
|  3|  may|
| 12|  dec|
+---+-----+



In [ ]:
#To remove the duplicated rows with the dataframe we use distinct()

df_new.distinct().show()

+---+-----+
| ID|Month|
+---+-----+
|  4|april|
|  1|  jan|
|  2|  feb|
|  3|  may|
|  5|  aug|
| 12|  dec|
|  6|  may|
+---+-----+



In [ ]:
df_new.show()

+---+-----+
| ID|Month|
+---+-----+
|  1|  jan|
|  2|  feb|
|  1|  jan|
|  3|  may|
|  4|april|
|  5|  aug|
|  6|  may|
|  6|  may|
|  3|  may|
| 12|  dec|
+---+-----+



In [ ]:
# To remove the duplicates from the month column
df_new.dropDuplicates(["month"]).show()
df_new.dropDuplicates(["ID"]).show()
df_new.dropDuplicates(["ID","Month"]).show()

+---+-----+
| ID|Month|
+---+-----+
|  3|  may|
|  2|  feb|
|  4|april|
|  1|  jan|
|  5|  aug|
| 12|  dec|
+---+-----+

+---+-----+
| ID|Month|
+---+-----+
|  1|  jan|
|  2|  feb|
|  3|  may|
|  4|april|
|  5|  aug|
|  6|  may|
| 12|  dec|
+---+-----+

+---+-----+
| ID|Month|
+---+-----+
|  4|april|
|  1|  jan|
|  2|  feb|
|  3|  may|
|  5|  aug|
| 12|  dec|
|  6|  may|
+---+-----+



In [ ]:
df_new.rdd.getNumPartitions()

2

In [ ]:
#To save the spark sql dataframe into a csv

df_new.write.csv("df_new1.csv",header=True)

In [ ]:
dfn=df_new.coalesce(1)

In [ ]:
dfn.rdd.getNumPartitions()

1

*************PART-B***********************

In [ ]:
# In this Module we are going to discuss about using SQL in Spark SQL

In [ ]:
leader_data = [["Dodge","Mohammed Saif"],["Cadillac", "George Carlin"],["BMW", "Stuart Broad"], ["Ford", "Abdalla"], ["Hyundai", "Chris Gayle"],["Lexus", "George Bush"], ["Mercury", "Tatyaso Martin"]]


In [ ]:
schema = StructType([StructField("Manufacturer", StringType()), StructField("SalesPerson", StringType())])

In [ ]:
df_lead=spark.createDataFrame(leader_data,schema=schema)

In [ ]:
df_lead.show()

+------------+--------------+
|Manufacturer|   SalesPerson|
+------------+--------------+
|       Dodge| Mohammed Saif|
|    Cadillac| George Carlin|
|         BMW|  Stuart Broad|
|        Ford|       Abdalla|
|     Hyundai|   Chris Gayle|
|       Lexus|   George Bush|
|     Mercury|Tatyaso Martin|
+------------+--------------+



In [ ]:
# To convert the dataframe into sql table

df_lead.createOrReplaceTempView("salestable")

In [ ]:
spark.sql("select * from salestable").show(10,False)

+------------+--------------+
|Manufacturer|SalesPerson   |
+------------+--------------+
|Dodge       |Mohammed Saif |
|Cadillac    |George Carlin |
|BMW         |Stuart Broad  |
|Ford        |Abdalla       |
|Hyundai     |Chris Gayle   |
|Lexus       |George Bush   |
|Mercury     |Tatyaso Martin|
+------------+--------------+



In [ ]:
spark.sql("select * from salestable where Manufacturer == 'Cadillac'").show(5,False)

+------------+-------------+
|Manufacturer|SalesPerson  |
+------------+-------------+
|Cadillac    |George Carlin|
+------------+-------------+



In [ ]:
# let us check to which all brand chris gayle promotes to

spark.sql("select * from salestable where SalesPerson=='Chris Gayle'").show()

+------------+-----------+
|Manufacturer|SalesPerson|
+------------+-----------+
|     Hyundai|Chris Gayle|
+------------+-----------+



In [ ]:
#To display the names of employees who name starts with G

spark.sql("select * from salestable where SalesPerson like 'G%'").show()

+------------+-------------+
|Manufacturer|  SalesPerson|
+------------+-------------+
|    Cadillac|George Carlin|
|       Lexus|  George Bush|
+------------+-------------+



In [ ]:
# Print the names of salesperson whose name is ending with a

spark.sql("select * from salestable where Salesperson like '%n'").show()

+------------+--------------+
|Manufacturer|   SalesPerson|
+------------+--------------+
|    Cadillac| George Carlin|
|     Mercury|Tatyaso Martin|
+------------+--------------+



END OF SALESTABLE

---



In [ ]:
df_car=spark.read.csv("/content/Cars_Sale.csv",header=True)

In [ ]:
df_car.show(1)

+------------+-------+------------+-------------+-----------+-----------+-------------+--------+---------+--------+
|Manufacturer|  Model|Vehicle_type|Latest_Launch|Units_Price|Units_Sold |Cost_incurred| Revenue|     Cost|  Profit|
+------------+-------+------------+-------------+-----------+-----------+-------------+--------+---------+--------+
|       Acura|Integra|   Passenger|   02-02-2012|       21.5|     16.919|        16.36|363.7585|276.79484|86.96366|
+------------+-------+------------+-------------+-----------+-----------+-------------+--------+---------+--------+
only showing top 1 row



In [ ]:
#To rename the column in the data frame

df_car=df_car.withColumnRenamed("Profit","profit")

In [ ]:
df_car.show()

+------------+-----------+------------+-------------+-----------+-----------+-------------+-----------+-----------+-----------+
|Manufacturer|      Model|Vehicle_type|Latest_Launch|Units_Price|Units_Sold |Cost_incurred|    Revenue|       Cost|     profit|
+------------+-----------+------------+-------------+-----------+-----------+-------------+-----------+-----------+-----------+
|       Acura|    Integra|   Passenger|   02-02-2012|       21.5|     16.919|        16.36|   363.7585|  276.79484|   86.96366|
|       Acura|         TL|   Passenger|   06-03-2011|       28.4|     39.384|       19.875|  1118.5056|    782.757|   335.7486|
|       Acura|         CL|   Passenger|   01-04-2012|      35.44|     14.114|       18.225|  500.20016|  257.22765|  242.97251|
|       Acura|         RL|   Passenger|   03-10-2011|         42|      8.588|       29.725|    360.696|   255.2783|   105.4177|
|        Audi|         A4|   Passenger|   10-08-2011|      23.99|     20.397|       22.255|  489.32403| 

In [ ]:
#Converting the pyspark.sql dataframe to SQLTABLE with name CARSDATA


df_car.createOrReplaceTempView("carsdata")

In [ ]:
spark.sql("select * from carsdata").show(100)

+------------+--------------+------------+-------------+-----------+-----------+-------------+-----------+-----------+-----------+
|Manufacturer|         Model|Vehicle_type|Latest_Launch|Units_Price|Units_Sold |Cost_incurred|    Revenue|       Cost|     profit|
+------------+--------------+------------+-------------+-----------+-----------+-------------+-----------+-----------+-----------+
|       Acura|       Integra|   Passenger|   02-02-2012|       21.5|     16.919|        16.36|   363.7585|  276.79484|   86.96366|
|       Acura|            TL|   Passenger|   06-03-2011|       28.4|     39.384|       19.875|  1118.5056|    782.757|   335.7486|
|       Acura|            CL|   Passenger|   01-04-2012|      35.44|     14.114|       18.225|  500.20016|  257.22765|  242.97251|
|       Acura|            RL|   Passenger|   03-10-2011|         42|      8.588|       29.725|    360.696|   255.2783|   105.4177|
|        Audi|            A4|   Passenger|   10-08-2011|      23.99|     20.397|   

In [ ]:
# To display the Acura as manufacturer

spark.sql("select * from carsdata where Manufacturer=='Acura'").show(100)

+------------+-------+------------+-------------+-----------+-----------+-------------+---------+---------+---------+
|Manufacturer|  Model|Vehicle_type|Latest_Launch|Units_Price|Units_Sold |Cost_incurred|  Revenue|     Cost|   profit|
+------------+-------+------------+-------------+-----------+-----------+-------------+---------+---------+---------+
|       Acura|Integra|   Passenger|   02-02-2012|       21.5|     16.919|        16.36| 363.7585|276.79484| 86.96366|
|       Acura|     TL|   Passenger|   06-03-2011|       28.4|     39.384|       19.875|1118.5056|  782.757| 335.7486|
|       Acura|     CL|   Passenger|   01-04-2012|      35.44|     14.114|       18.225|500.20016|257.22765|242.97251|
|       Acura|     RL|   Passenger|   03-10-2011|         42|      8.588|       29.725|  360.696| 255.2783| 105.4177|
+------------+-------+------------+-------------+-----------+-----------+-------------+---------+---------+---------+



In [ ]:
# To display the revenue genrated by each manufacturer

spark.sql("select Manufacturer,max(Revenue) from carsdata group by Manufacturer").show()

+------------+------------+
|Manufacturer|max(Revenue)|
+------------+------------+
|       Acura|   500.20016|
|        Audi|       85.56|
|         BMW|    681.8003|
|       Buick|     995.555|
|    Cadillac|  709.064925|
|   Chevrolet|   825.87827|
|    Chrysler|   895.86276|
|       Dodge|   961.06976|
|        Ford|  8836.53171|
|       Honda|   3544.3457|
|     Hyundai|  786.898908|
|    Infiniti|  698.703545|
|      Jaguar|    661.9876|
|        Jeep|   803.35422|
|       Lexus|   758.38836|
|     Lincoln|    977.9805|
|  Mercedes-B|      81.567|
|     Mercury|   760.90404|
|  Mitsubishi|  965.326912|
|      Nissan|  720.903892|
+------------+------------+
only showing top 20 rows



In [ ]:
spark.sql("select Manufacturer,max(Revenue) from carsdata group by Manufacturer order by Manufacturer").show()

+------------+------------+
|Manufacturer|max(Revenue)|
+------------+------------+
|       Acura|   500.20016|
|        Audi|       85.56|
|         BMW|    681.8003|
|       Buick|     995.555|
|    Cadillac|  709.064925|
|   Chevrolet|   825.87827|
|    Chrysler|   895.86276|
|       Dodge|   961.06976|
|        Ford|  8836.53171|
|       Honda|   3544.3457|
|     Hyundai|  786.898908|
|    Infiniti|  698.703545|
|      Jaguar|    661.9876|
|        Jeep|   803.35422|
|       Lexus|   758.38836|
|     Lincoln|    977.9805|
|  Mercedes-B|      81.567|
|     Mercury|   760.90404|
|  Mitsubishi|  965.326912|
|      Nissan|  720.903892|
+------------+------------+
only showing top 20 rows



In [ ]:
spark.sql("select Manufacturer,max(Revenue) from carsdata group by Manufacturer order by Manufacturer desc").show()

+------------+------------+
|Manufacturer|max(Revenue)|
+------------+------------+
|       Volvo|      86.498|
|  Volkswagen|       97.93|
|      Toyota|  969.354936|
|      Subaru|   663.69766|
|      Saturn|    861.4247|
|        Saab|    316.2015|
|     Porsche|     90.9056|
|     Pontiac|  853.873475|
|    Plymouth|     84.2592|
|  Oldsmobile|    931.0791|
|      Nissan|  720.903892|
|  Mitsubishi|  965.326912|
|     Mercury|   760.90404|
|  Mercedes-B|      81.567|
|     Lincoln|    977.9805|
|       Lexus|   758.38836|
|        Jeep|   803.35422|
|      Jaguar|    661.9876|
|    Infiniti|  698.703545|
|     Hyundai|  786.898908|
+------------+------------+
only showing top 20 rows



In [ ]:
# Performing Inner join of two tables carsdata and salestable

spark.sql("""select a.Manufacturer,a.Model,b.SalesPerson
from carsdata a
join salestable b
on trim(a.Manufacturer)= trim(b.Manufacturer)""").show(5,False)

+------------+---------+-------------+
|Manufacturer|Model    |SalesPerson  |
+------------+---------+-------------+
|Dodge       |Caravan  |Mohammed Saif|
|Dodge       |Durango  |Mohammed Saif|
|Dodge       |Dakota   |Mohammed Saif|
|Dodge       |Ram Van  |Mohammed Saif|
|Dodge       |Ram Wagon|Mohammed Saif|
+------------+---------+-------------+
only showing top 5 rows



In [ ]:
# Performing Inner join of two tables carsdata and salestable and where clause
spark.sql("""select a.Manufacturer,a.Model,b.SalesPerson
from carsdata a
join salestable b
on trim(a.Manufacturer)= trim(b.Manufacturer)
where trim(a.Manufacturer)='Cadillac'
""").show(5,False)

+------------+--------+-------------+
|Manufacturer|Model   |SalesPerson  |
+------------+--------+-------------+
|Cadillac    |Escalade|George Carlin|
|Cadillac    |Catera  |George Carlin|
|Cadillac    |Eldorado|George Carlin|
|Cadillac    |Seville |George Carlin|
|Cadillac    |DeVille |George Carlin|
+------------+--------+-------------+



In [ ]:
# saving the file
df_final = spark.sql("""select a.Manufacturer,a.Model,b.SalesPerson
from carsdata a
join salestable b
on trim(a.Manufacturer)= trim(b.Manufacturer)
where trim(a.Manufacturer)='Cadillac'
""")


In [101]:
df_final.coalesce(1).write.csv("vehicle1.csv",header=True)